In [1]:
import tensorflow as tf
from datetime import datetime
from tensorflow.keras.layers import *
import json
from tensorflowjs.converters.common import ARTIFACT_MODEL_TOPOLOGY_KEY
from tensorflowjs.converters.common import ARTIFACT_WEIGHTS_MANIFEST_KEY
from tensorflowjs.read_weights import read_weights
import common

%load_ext tensorboard

In [2]:
def resBlock(input, size, data_format, mul = None):
    negmul1 = negmul(input, size, mul)
    dconv1 = DepthwiseConv2D(
        3, strides=(1, 1),padding="SAME", data_format=data_format, use_bias=False
    )(negmul1)
    conv2 = Conv2D(size, kernel_size=(1,1), strides=(1,1), padding="SAME", data_format=data_format)(dconv1)
    out = Add()([negmul1, conv2])
    return out

In [3]:
def resBlock2(input, size, data_format, multiply: True, mul = None):
    negmul1 = negmul(input, size, mul)
    max_pool1 = MaxPool2D(
        pool_size=(2, 2), strides=(2, 2), padding='valid', data_format=data_format,
    )(negmul1)
    dconv1 = DepthwiseConv2D(
        3, strides=(2, 2),padding="SAME", data_format=data_format, use_bias=False
    )(negmul1)
    if(multiply):
        padding1 = tf.pad(max_pool1, [[0,0], [0,0], [0,0], [int(size/2), int(size/2)]])
        conv2 = Conv2D(size * 2, kernel_size=(1,1), strides=(1,1), padding="SAME", data_format=data_format)(dconv1)
        out = Add()([padding1, conv2])
    else:
        conv2 = Conv2D(size, kernel_size=(1,1), strides=(1,1), padding="SAME", data_format=data_format)(dconv1)
        out = Add()([max_pool1, conv2])
    return out

In [4]:
def resBlock3(input, size, data_format):
    dconv1_1 = DepthwiseConv2D(
        3, strides=(2, 2),padding="SAME", data_format=data_format, use_bias=False
    )(input)
    conv1_2 = Conv2D(size, kernel_size=(1,1), strides=(1,1), padding="SAME", data_format=data_format)(dconv1_1)
    return conv1_2

In [5]:
def resBlock4(input, size, data_format, mul = None, mul1 = None, mul2 = None):
    negmul1 = negmul(input, size, mul)
    newSize = int(size/4)
    conv1 = Conv2D(newSize, kernel_size=(1,1), strides=(1,1), padding="SAME", data_format=data_format)(negmul1)
    negmul2 = negmul(conv1, newSize, mul1)
    dconv1 = DepthwiseConv2D(
        3, strides=(1, 1),padding="SAME", data_format=data_format, use_bias=False
    )(negmul2)
    conv2 = Conv2D(newSize, kernel_size=(1,1), strides=(1,1), padding="SAME", data_format=data_format)(dconv1)
    add2 = Add()([negmul2, conv2])
    negmul3 = negmul(add2, newSize, mul2)
    return negmul3

In [6]:
def resBlock5(input, size, data_format, mul = None, mul1 = None, mul2 = None):
    negmul1 = negmul(input, size, mul)
    newSize = int(size/4)
    conv1 = Conv2D(newSize, kernel_size=(1,1), strides=(1,1), padding="SAME", data_format=data_format)(negmul1)
    res1 = resBlock(conv1, newSize, data_format, mul1)
    negmul2 = negmul(res1, newSize, mul2)
    conv2 = Conv2D(1, kernel_size=(3,3), strides=(1,1), padding="SAME", data_format=data_format)(negmul2)
    out = tf.math.sigmoid(conv2, name=None)
    return out

In [7]:
def negmul(input, size, mul = None):
    neg1 = input * -1
    relu1 = ReLU()(neg1)
    if mul is not None:
        var1 = tf.Variable(
            -1 * mul,
            name='qwe',
            trainable=True,
            shape=[1,1,size]
            )
    else:
        var1 = tf.Variable(
            tf.ones(
                [1,1,size], dtype=tf.dtypes.float32
            ),
            name='qwe',
            trainable=True,
            shape=[1,1,size]
            )
    relu2 = ReLU()(input)
    mul1 = Multiply()([relu1, var1])
    out = Add()([mul1, relu2])
    return out

In [8]:
def create_facenet(input_shape, batch_size = 1, output_dim=1404, data_format='channels_last', weight_dict={}):
    
    input_1 = Input(shape=input_shape, batch_size = batch_size, name="input_1", dtype="float32")
    conv1 = Conv2D(16, kernel_size=(3,3), strides=(2, 2), padding="SAME", data_format=data_format)(input_1)
    
    block1_1 = resBlock(conv1, 16, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu/Neg'])
    block1_2 = resBlock(block1_1, 16, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu_1/Neg'])
    block1_3 = resBlock2(block1_2, 16, data_format, True, weight_dict['StatefulPartitionedCall/model/p_re_lu_2/Neg'])

    block2_1 = resBlock(block1_3, 32, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu_3/Neg'])
    block2_2 = resBlock(block2_1, 32, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu_4/Neg'])
    block2_3 = resBlock2(block2_2, 32, data_format, True, weight_dict['StatefulPartitionedCall/model/p_re_lu_5/Neg'])

    block3_1 = resBlock(block2_3, 64, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu_6/Neg'])
    block3_2 = resBlock(block3_1, 64, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu_7/Neg'])
    block3_3 = resBlock2(block3_2, 64, data_format, True, weight_dict['StatefulPartitionedCall/model/p_re_lu_8/Neg'])

    block4_1 = resBlock(block3_3, 128, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu_9/Neg'])
    block4_2 = resBlock(block4_1, 128, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu_10/Neg'])
    block4_3 = resBlock2(block4_2, 128, data_format, False, weight_dict['StatefulPartitionedCall/model/p_re_lu_11/Neg'])

    block5_1 = resBlock(block4_3, 128, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu_12/Neg'])
    block5_2 = resBlock(block5_1, 128, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu_13/Neg'])


    negmul1 = negmul(block5_2, 128, weight_dict['StatefulPartitionedCall/model/p_re_lu_14/Neg'])
    max_pool1 = MaxPool2D(
        pool_size=(2, 2), strides=(2, 2), padding='valid', data_format=data_format,
    )(negmul1)

    blockb1_pre = resBlock3(negmul1, 128, data_format)
    blockb1 = Add()([max_pool1, blockb1_pre])
    
    const1 = tf.constant(
        [1, -1], dtype=None, shape=None, name='Const'
    )
    
    blockb1_1 = resBlock(blockb1, 128, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu_15/Neg'])
    blockb1_2 = resBlock(blockb1_1, 128, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu_16/Neg'])
    blockb1_3 = resBlock4(blockb1_2, 128, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu_17/Neg'], weight_dict['StatefulPartitionedCall/model/p_re_lu_18/Neg'], weight_dict['StatefulPartitionedCall/model/p_re_lu_19/Neg'])
    blockb1_4 = Conv2D(1404, kernel_size=(3,3), strides=(1,1), padding="SAME", data_format=data_format)(blockb1_3)
    blockb1_out = tf.reshape(
        blockb1_4, const1, name=None
    )
    
    blockb3_pre = resBlock3(negmul1, 128, data_format)
    blockb3 = Add()([max_pool1, blockb3_pre])
    blockb3_1 = resBlock(blockb3, 128, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu_20/Neg'])
    blockb3_2 = resBlock(blockb3_1, 128, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu_21/Neg'])
    blockb3_3 = resBlock4(blockb3_2, 128, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu_22/Neg'], weight_dict['StatefulPartitionedCall/model/p_re_lu_23/Neg'], weight_dict['StatefulPartitionedCall/model/p_re_lu_24/Neg'])
    blockb3_4 = Conv2D(266, kernel_size=(3,3), strides=(1,1), padding="SAME", data_format=data_format)(blockb3_3)
    blockb3_out = tf.reshape(
        blockb3_4, const1, name=None
    )
    
    blockb2_pre = resBlock3(negmul1, 128, data_format)
    blockb2 = Add()([max_pool1, blockb2_pre])
    blockb2_1 = resBlock5(blockb2, 128, data_format, weight_dict['StatefulPartitionedCall/model/p_re_lu_25/Neg'], weight_dict['StatefulPartitionedCall/model/p_re_lu_26/Neg'], weight_dict['StatefulPartitionedCall/model/p_re_lu_27/Neg'])
    blockb2_out = tf.reshape(
        blockb2_1, const1, name=None
    )
    

    return tf.keras.Model(inputs=[input_1], outputs=[blockb1_out, blockb2_out, blockb3_out])


In [9]:
def convertName(txt):
    txt = "StatefulPartitionedCall/model/" + txt
    txt = txt.replace("/kernel:0", "/Conv2D_weights")
    txt = txt.replace("/bias:0", "/Conv2D_bn_offset")
    txt = txt.replace("/depthwise_kernel:0", "/depthwise/ReadVariableOp")

    txt = txt.replace("conv2d_20/Conv2D_weights", "conv2d_20/Conv2D/ReadVariableOp")
    txt = txt.replace("conv2d_20/Conv2D_bn_offset", "conv2d_20/BiasAdd/ReadVariableOp")
    txt = txt.replace("conv2d_26/Conv2D_weights", "conv2d_26/Conv2D/ReadVariableOp")
    txt = txt.replace("conv2d_26/Conv2D_bn_offset", "conv2d_26/BiasAdd/ReadVariableOp")
    txt = txt.replace("conv2d_30/Conv2D_weights", "conv2d_30/Conv2D/ReadVariableOp")
    txt = txt.replace("conv2d_30/Conv2D_bn_offset", "conv2d_30/BiasAdd/ReadVariableOp")
    
    return txt
def restore_variables(model, weight_dict, data_format):
    for var in model.variables:
        try:
            var.assign(weight_dict[convertName(var.name)])
            print(var.name + ' is restored from ' + convertName(var.name))
        except: 
            print(var.name + ' is fail to restored from ' + convertName(var.name))
    return ''

In [10]:
def load_weight_from_graphModel(model_dir: str):
    model_file_path = model_dir + '/model.json'
    base_path = model_dir 
    with open(model_file_path, "r") as model_file:
        model_json = json.load(model_file)
    topology = model_json[ARTIFACT_MODEL_TOPOLOGY_KEY]
    weights_manifest = model_json[ARTIFACT_WEIGHTS_MANIFEST_KEY]
    weight_list = read_weights(weights_manifest, base_path, flatten=True)
    name, data = common.TFJS_NAME_KEY, common.TFJS_DATA_KEY
    weight_dict = dict((weight[name], weight[data]) for weight in weight_list)
    return weight_dict

In [11]:
def get_weight_dic_from_model(model):
    weight_dict = dict((v.name, i) for i, v in enumerate(model.variables))
    print(weight_dict)
    return weight_dict

In [12]:
weight_dict = load_weight_from_graphModel('./tfjs_model')
facemesh_tf = create_facenet((192,192,3), batch_size=None, output_dim=1404, data_format='channels_last', weight_dict=weight_dict)
restore_variables(facemesh_tf, weight_dict, data_format='channels_last')

2021-12-13 17:59:45.992727: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-12-13 17:59:46.004162: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fabc2aff640 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-12-13 17:59:46.004174: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


conv2d/kernel:0 is restored from StatefulPartitionedCall/model/conv2d/Conv2D_weights
conv2d/bias:0 is restored from StatefulPartitionedCall/model/conv2d/Conv2D_bn_offset
depthwise_conv2d/depthwise_kernel:0 is restored from StatefulPartitionedCall/model/depthwise_conv2d/depthwise/ReadVariableOp
conv2d_1/kernel:0 is restored from StatefulPartitionedCall/model/conv2d_1/Conv2D_weights
conv2d_1/bias:0 is restored from StatefulPartitionedCall/model/conv2d_1/Conv2D_bn_offset
depthwise_conv2d_1/depthwise_kernel:0 is restored from StatefulPartitionedCall/model/depthwise_conv2d_1/depthwise/ReadVariableOp
conv2d_2/kernel:0 is restored from StatefulPartitionedCall/model/conv2d_2/Conv2D_weights
conv2d_2/bias:0 is restored from StatefulPartitionedCall/model/conv2d_2/Conv2D_bn_offset
depthwise_conv2d_2/depthwise_kernel:0 is restored from StatefulPartitionedCall/model/depthwise_conv2d_2/depthwise/ReadVariableOp
conv2d_3/kernel:0 is restored from StatefulPartitionedCall/model/conv2d_3/Conv2D_weights
co

''

In [13]:
#print(weight_dict['StatefulPartitionedCall/model/conv2d_29/Conv2D_weights'])

[[[[ 6.29890859e-02  1.14021361e-01  1.95503294e-01 ...  9.74820703e-02
    -1.05236351e-01  2.37663835e-01]
   [ 4.51582223e-01 -3.06582749e-02  3.55894595e-01 ...  1.69941872e-01
     2.35512643e-03  4.13562879e-02]
   [-6.79290593e-01 -6.54211789e-02  1.14367478e-01 ... -1.62732959e-01
    -6.97381049e-02  4.47182685e-01]
   ...
   [-1.09425806e-01  1.18163638e-01  2.42153510e-01 ... -2.15807065e-01
    -1.50291190e-01  2.76762038e-01]
   [ 9.51402336e-02 -1.94328785e-01  2.70608738e-02 ... -1.82438880e-01
     6.51337396e-05 -2.14409187e-01]
   [ 2.19675526e-01 -1.59568384e-01  3.19005847e-02 ...  3.76959831e-01
     7.04126135e-02 -2.50225633e-01]]]]


In [14]:
#for i in weight_dict:
#    print(i)

StatefulPartitionedCall/model/conv2d/Conv2D_weights
StatefulPartitionedCall/model/conv2d/Conv2D_bn_offset
StatefulPartitionedCall/model/conv2d_1/Conv2D_weights
StatefulPartitionedCall/model/conv2d_1/Conv2D_bn_offset
StatefulPartitionedCall/model/conv2d_2/Conv2D_weights
StatefulPartitionedCall/model/conv2d_2/Conv2D_bn_offset
StatefulPartitionedCall/model/conv2d_3/Conv2D_weights
StatefulPartitionedCall/model/conv2d_19/Conv2D_weights
StatefulPartitionedCall/model/conv2d_3/Conv2D_bn_offset
StatefulPartitionedCall/model/conv2d_4/Conv2D_weights
StatefulPartitionedCall/model/conv2d_4/Conv2D_bn_offset
StatefulPartitionedCall/model/conv2d_5/Conv2D_weights
StatefulPartitionedCall/model/conv2d_19/Conv2D_bn_offset
StatefulPartitionedCall/model/conv2d_5/Conv2D_bn_offset
StatefulPartitionedCall/model/conv2d_6/Conv2D_weights
StatefulPartitionedCall/model/conv2d_6/Conv2D_bn_offset
StatefulPartitionedCall/model/conv2d_7/Conv2D_weights
StatefulPartitionedCall/model/conv2d_7/Conv2D_bn_offset
StatefulPart

In [15]:

facemesh_tf.save("./keras_models/facemesh_tf.h5")